In [14]:
from scipy.io import wavfile
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
import numpy
import IPython.display

In [5]:
samplerate, signal = wavfile.read("agua.wav")

In [6]:
winlen=0.025
winstep=0.01
numcep=13
nfilt=26
nfft=None
lowfreq=0
highfreq=None
preemph=0.97
ceplifter=22
appendEnergy=True,
winfunc=lambda x:numpy.ones((x,))

In [9]:
def calculate_nfft(samplerate, winlen):
    window_length_samples = winlen * samplerate
    nfft = 1
    while nfft < window_length_samples:
        nfft *= 2
    return nfft

In [11]:
nfft = nfft or calculate_nfft(samplerate, winlen)
nfft

512

In [13]:
highfreq= highfreq or samplerate/2
highfreq

8000.0

In [15]:
signal = numpy.append(signal[0],signal[1:]-preemph*signal[:-1])

In [23]:
import decimal
import math
def round_half_up(number):
    return int(decimal.Decimal(number).quantize(decimal.Decimal('1'), rounding=decimal.ROUND_HALF_UP))

def framesig(sig,frame_len,frame_step,winfunc=lambda x:numpy.ones((x,))):
    slen = len(sig)
    frame_len = int(round_half_up(frame_len))
    frame_step = int(round_half_up(frame_step))
    if slen <= frame_len:
        numframes = 1
    else:
        numframes = 1 + int(math.ceil((1.0*slen - frame_len)/frame_step))

    padlen = int((numframes-1)*frame_step + frame_len)

    zeros = numpy.zeros((padlen - slen,))
    padsignal = numpy.concatenate((sig,zeros))

    indices = numpy.tile(numpy.arange(0,frame_len),(numframes,1)) + numpy.tile(numpy.arange(0,numframes*frame_step,frame_step),(frame_len,1)).T
    indices = numpy.array(indices,dtype=numpy.int32)
    frames = padsignal[indices]
    win = numpy.tile(winfunc(frame_len),(numframes,1))
    return frames*win

In [28]:
frames = framesig(signal, winlen*samplerate, winstep*samplerate, winfunc)
frames[130]

array([-9.72700e+02, -7.36310e+02, -2.69120e+02,  4.81560e+02,
        1.30580e+03,  1.90513e+03,  2.16908e+03,  2.26109e+03,
        2.39382e+03,  2.52047e+03,  2.32080e+03,  1.53397e+03,
        2.98610e+02, -9.00970e+02, -1.65455e+03, -1.98667e+03,
       -2.23904e+03, -2.61502e+03, -2.89915e+03, -2.69540e+03,
       -1.87895e+03, -7.78680e+02,  1.35120e+02,  6.73820e+02,
        9.96330e+02,  1.32369e+03,  1.64378e+03,  1.74712e+03,
        1.48196e+03,  9.34200e+02,  3.47650e+02, -5.93200e+01,
       -2.32770e+02, -2.53320e+02, -2.57230e+02, -3.18990e+02,
       -3.80340e+02, -2.90220e+02,  3.79600e+01,  5.02250e+02,
        8.63430e+02,  9.78990e+02,  9.38250e+02,  9.02440e+02,
        8.62770e+02,  6.21180e+02,  7.56700e+01, -5.87640e+02,
       -1.06378e+03, -1.26454e+03, -1.38825e+03, -1.59645e+03,
       -1.75760e+03, -1.60914e+03, -1.11568e+03, -5.37050e+02,
       -9.01800e+01,  2.94430e+02,  8.06500e+02,  1.39551e+03,
        1.75632e+03,  1.69258e+03,  1.34827e+03,  1.010

In [29]:
def magspec(frames,NFFT):
    if numpy.shape(frames)[1] > NFFT:
        logging.warn('frame length (%d) is greater than FFT size (%d), frame will be truncated. Increase NFFT to avoid.', numpy.shape(frames)[1], NFFT)
    complex_spec = numpy.fft.rfft(frames,NFFT)
    return numpy.absolute(complex_spec)

In [32]:
pspec = 1.0/nfft * numpy.square(magspec(frames,nfft))
pspec

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [9.56675814e+02, 7.92005898e+02, 9.43119450e+00, ...,
        1.74556349e+00, 2.73048112e+00, 4.98126758e-01],
       [2.14232063e+02, 6.95901217e+01, 1.19809237e+01, ...,
        5.38843378e+00, 6.86140277e+00, 1.38107971e+01],
       [1.00997033e+03, 2.77178948e+02, 6.76299678e+00, ...,
        1.78606938e+00, 5.02586594e-01, 2.30856328e+00]])